# Nordrhein-Westfalen

Every federal state is represented by its own input directory and is processed into a NUTS level 2 directory containing a sub-folder for each discharge location. These folder names are derived from NUTS and reflect the CAMELS id. The NUTS level 2 code for Nordrhein-Westfalen is `DEA`.

To pre-process the data, you need to write (at least) two functions. One should extract all metadata and condense it into a single `pandas.DataFrame`. This is used to build the folder structure and derive the ids.
The second function has to take an id, as provided by the state authorities, called `provider_id` and return a `pandas.DataFrame` with the transformed data. The dataframe needs the three columns `['date', 'q' | 'w', 'flag']`.

For easier and unified output handling, the `camelsp` package contains a context object called `Bundesland`. It takes a number of names and abbreviations to identify the correct federal state and returns an object that holds helper and save functions.

The context saves files as needed and can easily be changed to save files with different strategies, ie. fill missing data with NaN, merge data into a single file, create files for each variable or pack everything together into a netcdf.

In [ ]:
import pandas as pd
import numpy as np
from pandas.errors import ParserError
import os
from pprint import pprint
from tqdm import tqdm
from typing import Union, Dict
from glob import glob
from datetime import datetime as dt
from dateparser import parse
import warnings
from io import StringIO

from camelsp import Bundesland

The context can also be instantiated as any regular Python class, ie. to load only the default input data path, that we will user later.

In [ ]:
# the context also makes the input path available, if camelsp was install locally
BASE = Bundesland('NRW').input_path
BASE

### Metadata reader

Define the function that extracts / reads and eventually merges all metadata for this federal state. You can develop the function here, without using the Bundesland context and then later use the context to pass extracted metadata. The Context has a function for saving *raw* metadata, that takes a `pandas.DataFrame` and needs you to identify the id column.
Here, *raw* refers to provider metadata, that has not yet been transformed into the CAMELS-de Metadata schema.

In [ ]:
# define the function 
def read_meta(base_path) -> pd.DataFrame:
    path = os.path.join(base_path, 'Stammdaten_CAMELS.xlsx')
    meta = pd.read_excel(path)
    return meta

# test it here
stamm = read_meta(BASE)

stamm

In [ ]:
# the id column will be ORT
id_column = 'ORT'

## file extract and parse

Here, we need to process the filename as the `'Ort'` is contained in the filename. Looks like the metadata header is **always** to line 32, indicating a finished header by `YTYP;`. Verify this.

In [ ]:
for fname in glob(os.path.join(BASE, 'Datenanfrage_CAMELS_*')):
    df = pd.read_csv(fname, encoding='latin1', sep=';', usecols=[0,1], nrows=32, header=None)
    if df.iloc[31, 0] != 'YTYP':
        print(fname)


That's will make our lifes way easier. Now go for all:

In [ ]:
# get all file names
filelist = glob(os.path.join(BASE, 'Datenanfrage_CAMELS_*'))

# container for meta-header and dataframes
meta = []
data = []

# go for each file
for fname in tqdm(filelist):
    # open
    with open(fname, 'rb') as f:
        txt = f.read().decode('latin1')
    
    # split header
    header = txt.splitlines()[:32]
    
    # build the meta by hand
    tups = [l.split(';') for l in header[:-1]]
    meta_dict = {t[0]: t[1] for t in tups}

    # check the parameter
    if meta_dict['Parameter'] == 'Wasserstand':
        variable = 'w'
    elif meta_dict['Parameter'] == 'Abfluss':
        variable = 'q'
    else:
        raise RuntimeError(f"Unknown Parameter: {meta_dict['Parameter']}")

    meta.append(meta_dict)
    
    # now get the body
    body = txt.splitlines()[32:]

    # now this stupid check
    second_header = [i for i, l in enumerate(body) if l.startswith('Station')]
    if len(second_header) > 0:
        # THERE IS A SECOND HEADER IN THE FILE !!!! come on!
        body = body[:second_header[0]]
    
    # write to buffer
    buffer = StringIO('\n'.join(body))
    buffer.seek(0)
    
    # read from memory
    df_data = pd.read_csv(buffer, sep=';', usecols=[0,1], skiprows=32, decimal=',', header=None, na_values='LUECKE', parse_dates=[0])
    
    df_data.columns = ['date', variable]
    df_data['flag'] = np.NaN
    
    # append
    data.append(df_data)
    
    
print(f"Parsed {len(meta)} metadata headers and {len(data)} data files")

That was really stupid. Ok. Check the metadata from the data files:

In [ ]:
extra = pd.DataFrame(meta).drop('Gewässer', axis=1)
extra

Now we have to left-join the data, as each Stationsnummer exists twice. Thus, it is only the combination of Stationsnummer and variable, that makes the data unique

In [ ]:
metadata = extra.drop('KOMMENTAR', axis=1).join(stamm.set_index(stamm.ORT.astype(str)), on='Stationsnummer', how='left')
metadata

In [ ]:
# build and id column
metadata['ID'] = metadata.apply(lambda r: r.Stationsnummer + '_' + r.Parameter, axis=1)

id_column = 'ID'

### Finally run

Now, the Q and W data can be extracted. The cool thing is, that all the id creation, data creation, merging and the mapping from our ids to the original ids and files is done by the context. This is helpful, as we less likely screw something up.

In [ ]:
with Bundesland('NRW') as bl:
    # save the metadata
    bl.save_raw_metadata(metadata, 'Stationsnummer', overwrite=True)

    # for reference, call the nuts-mapping as table
    nuts_map = bl.nuts_table
    print(nuts_map.head())

    
    with warnings.catch_warnings(record=True) as warns:
        for m, df in tqdm(zip(meta, data), total=len(meta)):
            # check the meta
            provider_id = str(m['Stationsnummer'])
            bl.save_timeseries(df, provider_id)

        # check if there were warnings (there are warnings)
        if len(warns) > 0:
            log_path = bl.save_warnings(warns)
            print(f"There were warnings during the processing. The log can be found at: {log_path}")
